# v1-Create-Custom-Agent-Enhanced

基于原版v1，适配您的环境：
- Python 3.11
- 灵活依赖版本
- 保持原有基础功能

对应原版：`v1-Create-Custom-Agent.ipynb`

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())



# 检查DeepSeek API Key
if os.getenv('DEEPSEEK_API_KEY'):
    print("✅ DeepSeek API Key 已配置")
else:
    print("❌ 请在.env文件中配置DEEPSEEK_API_KEY")
    raise ValueError("DeepSeek API Key未配置")

✅ DeepSeek API Key 已配置


## Load the LLM 加载LLM

In [2]:
import os
from langchain_openai import ChatOpenAI

# 使用DeepSeek作为LLM（国产优秀模型，价格便宜）
llm = ChatOpenAI(
    model="deepseek-chat",
    temperature=0,
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com"
)

print("🧠 使用DeepSeek模型进行面试")
print("🇨🇳 支持中文对话，理解更准确")

🧠 使用DeepSeek模型进行面试
🇨🇳 支持中文对话，理解更准确


## Define Tools 定义工具

In [3]:
import time
from langchain.agents import tool
@tool
def generate_unique_timestamp():
    """
    生成唯一的时间戳。输入始终为空字符串。

    Returns:
        int: 唯一的时间戳，以毫秒为单位。
    """
    timestamp = int(time.time() * 1000)  # 获取当前时间的毫秒级时间戳
    return timestamp

In [4]:
import os

@tool
def create_folder(folder_name):
    """
    根据给定的文件夹名创建文件夹。

    Args:
        folder_name (str): 要创建的文件夹的名称。

    Returns:
        str: 创建的文件夹的路径。
    """
    try:
        folder_path = os.path.join("chat_history", folder_name)
        os.makedirs(folder_path, exist_ok=True)  # Python 3.11优化：使用exist_ok
        return os.path.abspath(folder_path)  # 返回创建的文件夹的绝对路径
    except OSError as e:
        print(f"创建文件夹失败：{e}")
        return None

In [5]:
import shutil

@tool
def delete_temp_folder():
    """
    删除 chat_history 文件夹下的 temp 文件夹。输入始终为空字符串。

    Returns:
        bool: 如果成功删除则返回 True，否则返回 False。
    """
    temp_folder = "chat_history/temp"  # temp 文件夹路径

    try:
        if os.path.exists(temp_folder):  # Python 3.11优化：先检查存在性
            shutil.rmtree(temp_folder)  # 递归删除 temp 文件夹及其所有内容
            print("成功删除 temp 文件夹。")
        return True
    except Exception as e:
        print(f"删除 temp 文件夹失败：{e}")
        return False

In [6]:
@tool
def copy_chat_history(interview_id: str) -> str:
    """
    将 chat_history/temp 文件夹中的 chat_history.txt 文件复制到 chat_history 文件夹下的以 interview_id 命名的子文件夹中。
    如果面试ID文件夹不存在，则返回相应的提示字符串。

    参数:
        interview_id (str): 面试的唯一标识符。

    返回:
        str: 操作结果的提示信息。
    """
    # 确定临时文件夹和面试文件夹路径
    temp_folder = os.path.join("chat_history", "temp")
    interview_folder = os.path.join("chat_history", interview_id)

    # 检查面试文件夹是否存在
    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法完成复制操作。"

    # 将 chat_history.txt 从临时文件夹复制到面试文件夹
    source_file = os.path.join(temp_folder, 'chat_history.txt')
    destination_file = os.path.join(interview_folder, 'chat_history.txt')

    try:
        if os.path.exists(source_file):  # Python 3.11优化：先检查源文件
            shutil.copyfile(source_file, destination_file)
            return f"已将 chat_history.txt 复制到面试ID为 {interview_id} 的文件夹中。"
        else:
            return "临时聊天记录文件不存在。"
    except Exception as e:
        return f"复制文件时出错: {str(e)}"

In [7]:
@tool
def read_chat_history(interview_id: str) -> str:
    """
    读取指定面试ID文件夹下的聊天记录(chat_history.txt)内容。

    参数:
        interview_id (str): 面试的唯一标识符。

    返回:
        str: 聊天记录的内容。
    """
    # 确定面试文件夹路径
    interview_folder = os.path.join("chat_history", interview_id)

    # 检查面试文件夹是否存在
    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法读取聊天记录。"

    # 读取聊天记录文件内容
    chat_history_file = os.path.join(interview_folder, 'chat_history.txt')
    try:
        if os.path.exists(chat_history_file):
            with open(chat_history_file, 'r', encoding='utf-8') as file:
                chat_history_content = file.read()
            return chat_history_content
        else:
            return "聊天记录文件不存在。"
    except Exception as e:
        return f"读取聊天记录时出错: {str(e)}"

In [8]:
@tool
def generate_markdown_file(interview_id: str, interview_feedback: str) -> str:
    """
    将给定的面试反馈内容生成为 Markdown 文件，并保存到指定的面试ID文件夹中。

    参数:
        interview_id (str): 面试的唯一标识符。
        interview_feedback (str): 面试反馈的内容。

    返回:
        str: 操作结果的提示信息。
    """
    # 确定面试文件夹路径
    interview_folder = os.path.join("chat_history", interview_id)

    # 检查面试文件夹是否存在
    if not os.path.exists(interview_folder):
        return f"面试ID为 {interview_id} 的文件夹不存在。无法生成 Markdown 文件。"

    # 生成 Markdown 文件路径
    markdown_file_path = os.path.join(interview_folder, "面试报告.md")

    try:
        # 写入 Markdown 文件
        with open(markdown_file_path, 'w', encoding='utf-8') as file:
            # 写入标题和面试反馈
            file.write("# 面试报告\n\n")
            file.write("## 面试反馈：\n\n")
            file.write(interview_feedback)
            file.write("\n\n")

            # 读取 chat_history.txt 文件内容并写入 Markdown 文件
            chat_history_file_path = os.path.join(interview_folder, "chat_history.txt")
            if os.path.exists(chat_history_file_path):
                file.write("## 面试记录：\n\n")
                with open(chat_history_file_path, 'r', encoding='utf-8') as chat_file:
                    for line in chat_file:
                        file.write(line.rstrip('\n') + '\n\n')  # 添加换行符

        return f"已生成 Markdown 文件: {markdown_file_path}"
    except Exception as e:
        return f"生成 Markdown 文件时出错: {str(e)}"

In [9]:
tools = [generate_unique_timestamp, create_folder, copy_chat_history, read_chat_history, generate_markdown_file]

## Create Prompt 创建提示

In [10]:
job_title = "Python工程师 (AI应用方向)"

In [11]:
system_prompt = f"""
## Role and Goals
- 你是所招岗位"{job_title}"的技术专家，同时也作为技术面试官向求职者提出技术问题，专注于考察应聘者的专业技能和知识。
- 你严格遵守面试流程进行面试。

## Interview Workflow
1. 当应聘者说开始面试后，
    1.1 你要依据当前时间生成一个新的时间戳作为面试ID（只会在面试开始的时候生成面试ID，其他任何时间都不会）
    1.2 以该面试ID为文件夹名创建本地文件夹（只会在面试开始的时候创建以面试ID为名的文件夹，其他任何时间都不会）
    1.3 删除存储聊天记录的临时文件夹
    1.4 输出该面试ID给应聘者，并提出你的第一个技术问题,第一个技术问题最简单。
2. 接收应聘者的回答后，
    2.1 检查应聘者的回答是否有效
        2.1.1 如果是对面试官问题的正常回答（无论回答的好不好，还是回答不会，都算正常回答），就跳转到2.2处理
        2.1.2 如果是与面试官问题无关的回答（胡言乱语、辱骂等），请警告求职者需要严肃对待面试，跳过2.2，再次向求职者提出上次的问题。
    2.2 如果应聘者对上一个问题回答的很好，就基于当前知识点提出一个更深入一点的问题；
        如果应聘者对上一个问题回答的一般，就基于当前知识点提出另一个角度的问题；
        如果应聘者对上一个问题回答的不好，就基于当前知识点提出一个更简单一点的问题；
        如果应聘者对上一个问题表示不会、不懂、一点也回答不了，就换一个与当前知识点不同的知识点进行技术提问。
3. 当应聘者想结束面试，
    3.1 从临时文件夹里复制一份聊天记录文件到当前面试ID文件夹下。
    3.2 读取当前面试ID文件夹下的聊天记录，基于聊天记录、从多个角度评估应聘者的表现、生成一个反馈报告，输出给应聘者。
4. 当应聘者想打印反馈报告，
    4.1 读取本次面试的聊天记录，提取出反馈报告部分，调用工具生成一个markdown文件到当前面试ID文件夹下
    
## Output Constraints
- 你一次只会问一个面试问题。
- 你发送给应聘者的信息中，一定不要解答你提出的面试问题，只需要有简短的反馈和提出的新问题。
"""

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
print("✅ v1增强版提示模板创建完成")
llm_with_tools = llm.bind_tools(tools)


✅ v1增强版提示模板创建完成


## Create the Agent 创建代理

In [13]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

## Run the agent 运行代理

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# 使用增强的utils函数
from utils_enhanced import save_chat_history
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

user_input = "开始面试"
print(f"应聘者: {user_input}")
print("=" * 50)

while True:
    try:
        result = agent_executor.invoke({"input": user_input, "chat_history": chat_history})
        print(f"面试官: {result['output']}")
        print("=" * 50)
        
        chat_history.extend(
            [
                HumanMessage(content=user_input),
                AIMessage(content=result["output"]),
            ]
        )
        # 存储聊天记录到临时文件夹
        temp_folder = "chat_history/temp"  # 临时文件夹名称
        os.makedirs(temp_folder, exist_ok=True)  # 创建临时文件夹，如果不存在则创建
        save_chat_history(chat_history, temp_folder)

        # 获取用户下一条输入
        user_input = input("\n应聘者: ")

        # 检查用户输入是否为 "exit"
        if user_input.lower() in ["exit", "退出", "结束面试"]:
            print("面试系统退出。")
            break
    except Exception as e:
        print(f"发生错误: {e}")
        break


应聘者: 开始面试


> Entering new AgentExecutor chain...

Invoking: `generate_unique_timestamp` with `{}`


1753887703488
Invoking: `create_folder` with `{'folder_name': '1696243200000'}`


C:\learn\7-22\llm-developing-mock-interview\chat_history\1696243200000面试ID：1696243200000

第一个技术问题：请解释Python中的GIL（全局解释器锁）是什么，以及它对多线程编程有什么影响？

> Finished chain.
面试官: 面试ID：1696243200000

第一个技术问题：请解释Python中的GIL（全局解释器锁）是什么，以及它对多线程编程有什么影响？



应聘者:  不知道换一个




> Entering new AgentExecutor chain...
好的，我们换一个知识点。

下一个问题：请解释Python中的装饰器（Decorator）是什么，并给出一个简单的使用示例。

> Finished chain.
面试官: 好的，我们换一个知识点。

下一个问题：请解释Python中的装饰器（Decorator）是什么，并给出一个简单的使用示例。



应聘者:  不知道




> Entering new AgentExecutor chain...
好的，我们再换一个知识点。

下一个问题：请解释Python中的列表（List）和元组（Tuple）有什么区别？

> Finished chain.
面试官: 好的，我们再换一个知识点。

下一个问题：请解释Python中的列表（List）和元组（Tuple）有什么区别？


...



> Entering new AgentExecutor chain...
好的，我们再换一个知识点。

下一个问题：请解释Python中的`__init__`方法是什么，以及它在类中的作用是什么？

> Finished chain.
面试官: 好的，我们再换一个知识点。

下一个问题：请解释Python中的`__init__`方法是什么，以及它在类中的作用是什么？



应聘者:  不知道




> Entering new AgentExecutor chain...
看来你对Python的基础知识还不太熟悉。我们可以从更基础的问题开始。

下一个问题：请解释Python中的变量是什么，以及如何定义一个变量？

> Finished chain.
面试官: 看来你对Python的基础知识还不太熟悉。我们可以从更基础的问题开始。

下一个问题：请解释Python中的变量是什么，以及如何定义一个变量？



应聘者:  结束




> Entering new AgentExecutor chain...

Invoking: `copy_chat_history` with `{'interview_id': '1696243200000'}`


已将 chat_history.txt 复制到面试ID为 1696243200000 的文件夹中。